In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import os
import random

from sklearn.model_selection import train_test_split
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split

import pyarrow as pa
import pyarrow.parquet as pq

import math
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore, spearmanr , shapiro


In [3]:
CFG = {
    'BATCH_SIZE': 4096,
    'EPOCHS': 10,
    'LEARNING_RATE': 1e-3,
    'SEED' : 42
}
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(CFG['SEED']) # Seed 고정

In [5]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

/bin/bash: line 1: nvidia-smi: command not found


In [6]:
train= pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/CRT/train_part1.parquet' , engine= 'pyarrow')

In [8]:
import plotly.express as px

train.describe().T\
    .style.bar(subset=['mean'], color=px.colors.qualitative.G10[2])\
    .background_gradient(subset=['std'], cmap='Blues')\
    .background_gradient(subset=['50%'], cmap='BuGn')

,count,mean,std,min,25%,50%,75%,max
gender,3203885.000000,1.374313,0.477692,1.000000,1.000000,1.000000,2.000000,2.000000
age_group,3203885.000000,6.379988,1.587037,1.000000,6.000000,7.000000,7.000000,8.000000
inventory_id,3203885.000000,31.559231,21.964794,2.000000,11.000000,36.000000,42.000000,95.000000
day_of_week,3203885.000000,4.150190,1.795642,1.000000,3.000000,4.000000,6.000000,7.000000
hour,3203885.000000,12.589877,6.327867,0.000000,8.000000,12.000000,18.000000,23.000000
l_feat_1,3203885.000000,1.841406,0.370378,1.000000,2.000000,2.000000,2.000000,2.000000
l_feat_2,3203885.000000,1.828081,0.379389,1.000000,2.000000,2.000000,2.000000,2.000000
l_feat_3,3203885.000000,2.333113,0.741038,1.000000,2.000000,2.000000,3.000000,3.000000
l_feat_4,3203885.000000,10.027688,5.634881,1.000000,7.000000,7.000000,14.000000,26.000000
l_feat_5,3203885.000000,415.172180,295.095001,1.000000,132.000000,383.000000,675.000000,1079.000000


In [9]:
def summary(df):
    sum = pd.DataFrame(train.dtypes, columns=['dtypes'])
    sum['결측치 개수'] = df.isna().sum()
    sum['결측치 비율'] = (df.isna().sum())/len(df)
    sum['고유값 개수'] = df.nunique().values
    sum['데이터 개수'] = df.count().values


    return sum

summary(train).style.background_gradient(cmap='Blues')

,dtypes,결측치 개수,결측치 비율,고유값 개수,데이터 개수
gender,float32,0,0.000000,2,3203885
age_group,float32,0,0.000000,8,3203885
inventory_id,float32,0,0.000000,18,3203885
day_of_week,float32,0,0.000000,7,3203885
hour,float32,0,0.000000,24,3203885
seq,object,0,0.000000,2716557,3203885
l_feat_1,float32,0,0.000000,2,3203885
l_feat_2,float32,0,0.000000,2,3203885
l_feat_3,float32,0,0.000000,3,3203885
l_feat_4,float32,0,0.000000,26,3203885


In [10]:
corr = train.corr(numeric_only= True)

corr = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
corr.style.background_gradient(cmap='Blues')

/usr/local/lib/python3.12/dist-packages/pandas/io/formats/style.py:3807: RuntimeWarning: All-NaN slice encountered
  smin = np.nanmin(gmap) if vmin is None else vmin
/usr/local/lib/python3.12/dist-packages/pandas/io/formats/style.py:3808: RuntimeWarning: All-NaN slice encountered
  smax = np.nanmax(gmap) if vmax is None else vmax


feature 분석

(A) 범주형(임베딩용)

- user/context: gender, age_group, day_of_week, hour

- item: inventory_id

- sequence: seq (쉼표로 구분된 과거 inventory_id 시퀀스)

<br>


(B) 연속형(스케일링/버킷화)

- l_feat_1 ~ l_feat_27 : 로그 기반/랭크류처럼 보이는 long-term 통계 → 표준화(μ,σ) 또는 분위수 변환 권장

- feat_e_1 ~ feat_e_10, feat_d_1 ~ feat_d_6, feat_c_1 ~ feat_c_8, feat_b_1 ~ feat_b_6,
feat_a_1 ~ feat_a_18 : 아이템/유저/컨텍스트 파생 수치 → 표준화 또는 버킷화(DeepFM 필드로 쓸 거면 버킷)

- history_a_1 ~ history_a_7, history_b_1 ~ history_b_30 : 시퀀스 통계 요약(정규화된 비율/가중치처럼 보임) → 그대로 연속형으로 사용(스케일만 맞춤)

